# Basic Pandas operations

**Goal**: Our goal here is to learn how to load a dataset into a Pandas DataFrame. The dataset can come either in CSV or in JSON format. We will see also how to perform basic data manipulations and very basic data visualizations so that you understand the nature of your data.

## 1. Loading a dataset in CSV format

In [ ]:
import pandas as pd
data = pd.read_csv('../data/pandas_tutorial_read.csv') 
data.head()

Is the above correct? Most likely not. We see there are ';' and the data seem to be in one column. The default delimiter is ',' so we need to change it.

In [ ]:
data = pd.read_csv('../data/pandas_tutorial_read.csv', delimiter=';') 
data.head()

This looks better. But something else does not look good now. Now we don't have a header for the data. This usually comes with the documentation. I provide it here.

In [ ]:
data = pd.read_csv('../data/pandas_tutorial_read.csv', 
                   delimiter=';', 
                   names = ['my_datetime', 'event', 'country', 'user_id', 'source', 'topic']) 
data.head()

You don't have to have the data locally you can also load a csv file from a url. In this case, it is not stored locally though. This is especially useful if you are using colab, in this case you can copy the url of the raw data in github and load the data in pandas directly from github.

```python
url = "some url"
data = pd.read_csv(url, 
                   delimiter=';', 
                   names = ['my_datetime', 'event', 'country', 'user_id', 'source', 'topic']) 
data.head()

```

In [ ]:
url = "https://raw.githubusercontent.com/ahmadajal/DM_ML_course_public/master/2.%20Data%26EDA/data/pandas_tutorial_read.csv"
data = pd.read_csv(url, 
                   delimiter=';', 
                   names = ['my_datetime', 'event', 'country', 'user_id', 'source', 'topic']) 
data.head()

You can see 

- whole dataset: ```data```
- the beginning ```head()``` 
- the ```tail()``` or 
- a ```sample(5)```

Try it out.

## DataFrame components
There are three components of the DataFrame: the index, columns and data (values). We can extract each of these components into their own variables. Let's do that and then inspect them:

In [ ]:
index = data.index
columns = data.columns
values = data.values

In [ ]:
index

In [ ]:
columns

In [ ]:
values

## Data types of the components

In [ ]:
type(index)

In [ ]:
type(columns)

In [ ]:
type(values)


The index and the columns are the same type: a pandas **`Index`** object (**`RangeIndex`** is of type **`Index`**), which is a sequence of labels for either the rows or the columns.

The values are a NumPy **`ndarray`**, which stands for n-dimensional array, and is the primary container of data in the NumPy library. Pandas is built directly on top of NumPy.

### Selecting columns

If you want to select two particular columns you can do it like that:

```data[['country', 'user_id']]``` 

or you can take the columns in a different order: 

```data[['user_id', 'country']]```.

The way to remember the syntax is that outer brackets signify that you want to select columns, and the inner brackets are for the list itself.

Try it out.

The above returns a pandas.DataFrame. If you want to return a pandas.Series instead then you can use this syntax:

```data.user_id ```

or 

``` data['user_id'] ```

If you want to filter one those users that came from SEO then you can write:

``` data[data.source == 'SEO'] ```

where the inner statement will create a boolean mask.

### Chaining

You can combine multiple selection methods as follows:

``` data.head()[['country', 'user_id']] ```

**CAUTION**: A thing to keep in mind is that when you use chaining you work on *copies* of the original DataFrame. So if you use chaining to change data, you may observe that the original DataFrame was not changed.

---

Now it's your turn to solve an exercise and deepen your knowledge.


<div class="alert alert-block alert-success">
    <h2>Exercise 1:</h2>


    
>Select the user_id, the country and the topic columns for the users who are from country_2, and show only the first 10 rows
</div>

In [ ]:
# enter your solution here.

In [ ]:

# possible solution 
# data[['user_id', 'topic', 'country']][data['country'] == 'country_2'].head(10)

---
## 2. Loading JSON files

Many of the data in the Internet exists in JSON format which is a structured text format, and is very similar to a Python dictionary.

We will see how to load a JSON dataset in a Pandas DataFrame.

We will use the Citibike API that provides a real-time view of the Citibike stations in New York.
The API call at http://www.citibikenyc.com/stations/json.

In [ ]:
import requests
url = 'http://www.citibikenyc.com/stations/json'
data = requests.get(url).json()
data

Above you see how the JSON file looks. The JSON results contain two keys: The `executionTime` and `stationBeanList`. The `stationBeanList` is a list of dictionaries, with each dictionary corresponding to a Citibike station.

In [ ]:
data.keys()

With Pandas we can easily convert a list of dictionaries into a DataFrame

In [ ]:
import pandas
df = pandas.DataFrame(data["stationBeanList"])
df.head(5)

Let's try to understand the columns:

We notice that:

- **totalDocks** = **availableBikes** (bikes ready to rent) + **availableDocks** (how many docks are free)

To see if the data has been imported correctly, we can verify the datatypes of the columns. Pandas tries to infer the datatypes and for this case it does a pretty good job. In general, you should consider providing explicitly the datatypes of each column.

In [ ]:
df.dtypes

One column that looks not parsed correctly is the **lastCommunicationTime** which is an `object` (i.e., `string`), so you may want to convert it to the `datetime` type.

<div class="alert alert-block alert-success">
    <h2>Exercise 2:</h2>


    
>Convert the **lastCommunicationTime** into a `datetime` datatype. <br>
**Hint**: Use the [pandas.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) function.
</div>

In [ ]:
# Your solution here

In [ ]:
# df["lastCommunicationTime"] = pd.to_datetime(df["lastCommunicationTime"])
# df.head()

Let's confirm that the **lastCommunicationTime** column is of type `datetime`.

### Adding a column

We can add a column `perc_full` that shows how full is each station.

In [ ]:
df["perc_full"] = df['availableBikes']/df['totalDocks']
df.head()

## Plotting

Now we can do some basic visualizations to get a feeling of the data.

In [ ]:
# some basic styling and importing of the right styles
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 10)
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])

### Histograms

Let's start with a histogram of the sizes of the bike docks.

In [ ]:
df["totalDocks"].hist(alpha=0.75);
# alpha indicates the transparency of the bars

Therefore, most of the bike stations have places for around 30 bikes.
Now, let's do a histogram of the available bikes across bike stations (real-time)

In [ ]:
df["availableBikes"].hist(alpha=0.75);

Now we will plot a histogram of how full each station is using the column we previously created `df["perc_full"]`

In [ ]:
df["perc_full"].hist(alpha=0.5, bins=20);

### Scatter Plot

A scatter plot is useful for visualizing two variables at the same time. They are usually used to understand *correlations* between variables, but now we will use it as an elementary plotting function, instead of plotting directly on the map which is more complicated. 

We will use the longitude and latitude coordinates to plot the location of points.

In [ ]:
df.plot(
    kind='scatter', 
    x='longitude', 
    y='latitude',
    figsize = (10,10)
);

<div class="alert alert-block alert-success">
    <h2>Exercise 3:</h2>

<br>    
Adjust the above graph to convey more information. Now the size of the marker in the scatterplot will communicate how full each station is. <br>
**HINT:** The *s* parameter in the plot command controls the size of the market. We will make the size proportional to the perc_full column
</div>

In [ ]:
# your solution here

**Solution:**

In [ ]:
# The s parameter controls the size of the market. We make the size proportional to the perc_full column
df.plot(
    kind='scatter', 
    x='longitude', 
    y='latitude', 
    s = 100*df['perc_full'], alpha=0.75,
    figsize = (10,10)
)

## Summary Statistics

You can also use the `describe` function of Pandas to get some general understanding of the central values and the tendencies of each column.

In [ ]:
df.describe()

Now you should ask yourself. **Are the values in the summary statistics what you expected them to be?**

---
## Writing the data to a CSV

With the above, we just scratched the surface of what it means to do data processing.

After you did your basic data processing, you may want to save the DataFrame in a new CSV file, so that you don't have to repeat the same pre-processing everytime. You can use the [to_csv](https://datatofish.com/export-dataframe-to-csv/) function.

In [ ]:
# uncomment the following to save the file
# df.to_csv("my_new_file.csv", sep=",")